<a href="https://colab.research.google.com/github/Keita062/Text_Analysis_Charenge/blob/main/morphological%20analysis(%E5%BD%A2%E6%85%8B%E7%B4%A0%E8%A7%A3%E6%9E%90).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:

f = open("/content/演説全文.txt", "r")
content = f.read()
f.close()


# ファイルを読み込み
with open("/content/演説全文.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# 改行や空白を取り除く
lines = [line.strip() for line in lines]

# DataFrameに変換
speach = pd.DataFrame(lines, columns=["speech_line"])

参考ページ

https://github.com/Hironsan/natural-language-preprocessings/blob/master/preprocessings/ja/tokenizer.py

In [18]:
!pip install janome


In [12]:
# -*- coding: utf-8 -*-
from collections import namedtuple

import MeCab
from janome.tokenizer import Tokenizer


class JanomeTokenizer(object):

    def __init__(self, user_dic_path='', user_dic_enc='utf8'):
        self._t = Tokenizer(udic=user_dic_path, udic_enc=user_dic_enc)

    def wakati(self, sent):
        words = [token.surface for token in self.tokenize(sent)]
        return words

    def wakati_baseform(self, sent):
        words = [token.base_form if token.base_form != '*' else token.surface
                 for token in self.tokenize(sent)]
        return words

    def tokenize(self, sent):
        token = namedtuple('Token', 'surface, pos, pos_detail1, pos_detail2, pos_detail3,\
                                             infl_type, infl_form, base_form, reading, phonetic')
        for t in self._t.tokenize(sent):
            poses = t.part_of_speech.split(',')
            yield token(t.surface, poses[0], poses[1], poses[2], poses[3],
                        t.infl_type, t.infl_form, t.base_form, t.reading, t.phonetic)

    def filter_by_pos(self, sent, pos=('名詞', )):
        tokens = [token for token in self.tokenize(sent) if token.pos in pos]
        return tokens


class MeCabTokenizer(object):

    def __init__(self, user_dic_path='', sys_dic_path=''):
        option = ''
        if user_dic_path:
            option += ' -u {0}'.format(user_dic_path)
        if sys_dic_path:
            option += ' -d {0}'.format(sys_dic_path)
        self._t = MeCab.Tagger(option)

    def wakati(self, sent):
        words = [token.surface for token in self.tokenize(sent)]
        return words

    def wakati_baseform(self, sent):
        words = [token.base_form if token.base_form != '*' else token.surface
                 for token in self.tokenize(sent)]
        return words

    def tokenize(self, text):
        self._t.parse('')
        chunks = self._t.parse(text.rstrip()).splitlines()[:-1]  # Skip EOS
        token = namedtuple('Token', 'surface, pos, pos_detail1, pos_detail2, pos_detail3,\
                                                                         infl_type, infl_form, base_form, reading, phonetic')
        for chunk in chunks:
            if chunk == '':
                continue
            surface, feature = chunk.split('\t')
            feature = feature.split(',')
            if len(feature) <= 7:  # 読みがない
                feature.append('')
            if len(feature) <= 8:  # 発音がない
                feature.append('')
            yield token(surface, *feature)

    def filter_by_pos(self, sent, pos=('名詞',)):
        tokens = [token for token in self.tokenize(sent) if token.pos in pos]
        return tokens

In [15]:
Janome_Tokenize = JanomeTokenizer()

In [16]:
speach["wakati"] = speach["speech_line"].apply(Janome_Tokenize.wakati)

In [17]:
speach.head()

,speech_line,wakati
0,演説全文,"[演説, 全文]"
1,一 はじめに,"[一, , はじめ, に]"
2,,[]
3,この度、第百二代内閣総理大臣に就任いたしました。,"[この, 度, 、, 第, 百, 二, 代, 内閣, 総理, 大臣, に, 就任, いたし,..."
4,「すべての人に安心と安全を」,"[「, すべて, の, 人, に, 安心, と, 安全, を, 」]"
